# PySDD: SDD Python Wrapper

*Wannes Meert (KU Leuven), Arthur Choi (UCLA) - 2018*

In [ ]:
import os, sys
sys.path.append(os.pardir)

In [ ]:
from pysdd.sdd import Vtree, SddManager, WmcManager, Fnf

In [ ]:
from array import array
from graphviz import Source
import math

## Vtree

In [ ]:
vtree = Vtree(4)

In [ ]:
print("Size: {}".format(vtree.size()))

In [ ]:
Source(vtree.dot())

## SDD Manager

Ask for documentation:

In [ ]:
SddManager?

Create new manager with four variables A, B, C, and D

In [ ]:
mgr = SddManager(var_count=4)
a, b, c, d = [mgr.literal(i) for i in range(1,5)]
str(a)

In [ ]:
mgr.is_var_used(-2)

Create the formula $(A \land B) \lor (C \land D)$

In [ ]:
fab   = mgr.conjoin(a, b)
fcd   = mgr.conjoin(c, d)
fabcd = mgr.disjoin(fab, fcd)
str(fcd)

In [ ]:
Source(fabcd.dot())

Use shorter syntax by mapping and/or to */+

In [ ]:
mgr = SddManager(var_count=4)
a, b, c, d = [mgr.literal(i) for i in range(1,5)]

In [ ]:
fabcd = (a*b) + (c*d)
Source(fabcd.dot())

## Examples

### Example: API documentation example 1

In [ ]:
var_count = 4
var_order = array("l", [2,1,4,3])
tree_type = "balanced"

In [ ]:
vtree = Vtree(var_count=var_count, var_order=var_order, type=tree_type.encode())

In [ ]:
Source(vtree.dot())

In [ ]:
mgr = SddManager(vtree=vtree)

construct a formula $(A \land B) \lor (B \land C) \lor (C \land D)$

In [ ]:
f_a = mgr.literal(1)
f_b = mgr.literal(2)
f_c = mgr.literal(3)
f_d = mgr.literal(4)

In [ ]:
str(f_a)

In [ ]:
alpha = mgr.false()

In [ ]:
beta  = mgr.conjoin(f_a,f_b)
alpha = mgr.disjoin(alpha,beta)
beta  = mgr.conjoin(f_b,f_c)

In [ ]:
Source(mgr.dot(beta))

In [ ]:
alpha = mgr.disjoin(alpha,beta)
beta  = mgr.conjoin(f_c,f_d)
alpha = mgr.disjoin(alpha,beta)

In [ ]:
Source(alpha.dot())

### Example: Load from CNF file

In [ ]:
mgr = SddManager()
cnf_str = """p cnf 6 3
1 4 0
-1 2 5 0
-1 -2 3 6 0
"""
f = mgr.cnf_from_string(cnf_str)

In [ ]:
Source(f.dot())

#### Change the SDD after loading the CNF

In [ ]:
print(f"Var count before: {mgr.var_count()}")
mgr.add_var_after_last()
mgr.add_var_after_last()
print(f"Var count after:  {mgr.var_count()}")

In [ ]:
g = mgr.literal(7)
h = mgr.literal(8)

fp = mgr.conjoin(g, h)
Source(fp.dot())

In [ ]:
fpp = mgr.disjoin(f, fp)
Source(fpp.dot())

### Example: API documentation example 6

In [ ]:
vtree = Vtree(filename="sdd/lib/sdd-1.1.1/examples/input/opt-swap.vtree".encode())
Source(vtree.dot())

In [ ]:
mgr = SddManager(vtree=vtree)
mgr.initialize_search_state()
alpha = mgr.read("sdd/lib/sdd-1.1.1/examples/input/opt-swap.sdd".encode())
print(f"Size alpha: {alpha.size()}")

In [ ]:
Source(alpha.dot())

In [ ]:
alpha.ref()

Minimize the SDD

In [ ]:
mgr.minimize()
print(f"Size alpha: {alpha.size()}")

In [ ]:
Source(alpha.dot())

Show the new vtree after minimization.

In [ ]:
new_vtree = mgr.vtree()
Source(new_vtree.dot())

Augment the SDD

In [ ]:
beta = alpha * (mgr.literal(4) + mgr.literal(5))
print(f"Size beta: {beta.size()}")

In [ ]:
Source(beta.dot())

Perform minimization again on new SDD

In [ ]:
beta.ref()

In [ ]:
mgr.minimize()
print(f"Size: {beta.size()}")

In [ ]:
Source(beta.dot())

In [ ]:
beta.deref()

### Example: WMC

In [ ]:
mgr = SddManager(var_count=4)
a, b, c, d = [mgr.literal(i) for i in range(1,5)]
f = (a*b) + (c*d)

In [ ]:
#f.is_literal() # TODO: this crashes

In [ ]:
wmc = f.wmc(log_mode=True)
math.exp(wmc.literal_weight(a.literal)), math.exp(wmc.literal_weight(0))

In [ ]:
w = wmc.propagate()
print(f"Number of models: {int(math.exp(w))}")

In [ ]:
wmc.set_literal_weight(a.literal, math.log(0.5))
wmc.set_literal_weight(-a.literal, math.log(0.5))

In [ ]:
w = wmc.propagate()
print(f"Weighted model count: {math.exp(w)}")

## Conclusion